In [5]:
# !pip install faker
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 21.4 MB/s eta 0:00:0000:0100:01


In [12]:
import mysql.connector
import random
import faker
from datetime import datetime, timedelta

# Create a Faker instance to generate fake data
fake = faker.Faker()

# Connect to your MySQL database
conn = mysql.connector.connect(
    host="server.vishalpandey.co.in",  # Adjust as per your settings
    user="vishal",
    password="vishal123",
    database="db",
    port=6500

)
cur = conn.cursor()

# Function to insert data into the customers table
def insert_customers(num_records):
    for _ in range(num_records):
        customer_type = random.choice(['individual', 'business'])
        if customer_type == 'individual':
            full_name = fake.name()
            date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=70)
            pan_number = fake.bothify(text='???######?')
            business_name = None
        else:
            full_name = 'N/A'
            date_of_birth = None
            pan_number = fake.bothify(text='???######?')
            business_name = fake.company()
        
        address = fake.address()
        phone_number = "9898989898"
        email = fake.email()

        cur.execute("""
            INSERT INTO customers (customer_type, full_name, date_of_birth, pan_number, business_name, address, phone_number, email)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (customer_type, full_name, date_of_birth, pan_number, business_name, address, phone_number, email))

# Function to insert data into financial_institutions table
def insert_financial_institutions(num_records):
    for _ in range(num_records):
        institution_name = fake.company()
        institution_type = random.choice(['Bank', 'NBFC', 'Credit Union'])
        address = fake.address()
        contact_number = "9999999999"
        email = fake.company_email()

        cur.execute("""
            INSERT INTO financial_institutions (institution_name, institution_type, address, contact_number, email)
            VALUES (%s, %s, %s, %s, %s)
        """, (institution_name, institution_type, address, contact_number, email))

# Function to insert data into credit_accounts table
def insert_credit_accounts(num_records):
    cur.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cur.fetchall()]

    cur.execute("SELECT institution_id FROM financial_institutions")
    institution_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        customer_id = random.choice(customer_ids)
        institution_id = random.choice(institution_ids)
        account_type = random.choice(['loan', 'credit card'])
        account_number = fake.bothify(text='??##########')
        
        if account_type == 'credit card':
            credit_limit = random.randint(50000, 500000)
            loan_amount = None
            outstanding_balance = random.randint(0, credit_limit)
        else:
            credit_limit = None
            loan_amount = random.randint(50000, 3000000)
            outstanding_balance = random.randint(0, loan_amount)
        
        account_status = random.choice(['active', 'closed', 'defaulted'])
        opened_date = fake.date_between(start_date='-5y', end_date='today')
        closed_date = opened_date + timedelta(days=random.randint(100, 1500)) if account_status == 'closed' else None

        cur.execute("""
            INSERT INTO credit_accounts (customer_id, institution_id, account_type, account_number, credit_limit, loan_amount, outstanding_balance, account_status, opened_date, closed_date)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (customer_id, institution_id, account_type, account_number, credit_limit, loan_amount, outstanding_balance, account_status, opened_date, closed_date))

# Function to insert data into payments table
def insert_payments(num_records):
    cur.execute("SELECT account_id FROM credit_accounts")
    account_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        account_id = random.choice(account_ids)
        payment_date = fake.date_between(start_date='-2y', end_date='today')
        payment_amount = random.randint(1000, 50000)
        status = random.choice(['paid', 'late', 'missed'])

        cur.execute("""
            INSERT INTO payments (account_id, payment_date, payment_amount, status)
            VALUES (%s, %s, %s, %s)
        """, (account_id, payment_date, payment_amount, status))

# Function to insert data into credit_scores table
def insert_credit_scores(num_records):
    cur.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        customer_id = random.choice(customer_ids)
        credit_score = random.randint(300, 900)
        score_date = fake.date_this_year()

        cur.execute("""
            INSERT INTO credit_scores (customer_id, credit_score, score_date)
            VALUES (%s, %s, %s)
        """, (customer_id, credit_score, score_date))

# Function to insert data into credit_reports table
def insert_credit_reports(num_records):
    cur.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        customer_id = random.choice(customer_ids)
        report_date = fake.date_this_year()
        report_data = fake.text()
        credit_score = random.randint(300, 900)

        cur.execute("""
            INSERT INTO credit_reports (customer_id, report_date, report_data, credit_score)
            VALUES (%s, %s, %s, %s)
        """, (customer_id, report_date, report_data, credit_score))

# Function to insert data into public_records table
def insert_public_records(num_records):
    cur.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        customer_id = random.choice(customer_ids)
        record_type = random.choice(['bankruptcy', 'court_judgment'])
        record_details = fake.text()
        record_date = fake.date_between(start_date='-10y', end_date='today')

        cur.execute("""
            INSERT INTO public_records (customer_id, record_type, record_details, record_date)
            VALUES (%s, %s, %s, %s)
        """, (customer_id, record_type, record_details, record_date))

# Function to insert data into disputes table
def insert_disputes(num_records):
    cur.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cur.fetchall()]

    cur.execute("SELECT account_id FROM credit_accounts")
    account_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        customer_id = random.choice(customer_ids)
        account_id = random.choice(account_ids)
        dispute_description = fake.sentence()
        status = random.choice(['open', 'resolved'])
        filed_date = fake.date_this_year()
        resolved_date = filed_date + timedelta(days=random.randint(10, 90)) if status == 'resolved' else None

        cur.execute("""
            INSERT INTO disputes (customer_id, account_id, dispute_description, status, filed_date, resolved_date)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (customer_id, account_id, dispute_description, status, filed_date, resolved_date))

# Function to insert data into loan_collateral table
def insert_loan_collateral(num_records):
    cur.execute("SELECT account_id FROM credit_accounts WHERE account_type = 'loan'")
    loan_account_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        account_id = random.choice(loan_account_ids)
        collateral_type = random.choice(['real estate', 'vehicle', 'jewelry'])
        collateral_value = random.randint(100000, 2000000)

        cur.execute("""
            INSERT INTO loan_collateral (account_id, collateral_type, collateral_value)
            VALUES (%s, %s, %s)
        """, (account_id, collateral_type, collateral_value))

# Function to insert data into institution_reports table
def insert_institution_reports(num_records):
    cur.execute("SELECT institution_id FROM financial_institutions")
    institution_ids = [row[0] for row in cur.fetchall()]

    cur.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cur.fetchall()]

    cur.execute("SELECT account_id FROM credit_accounts")
    account_ids = [row[0] for row in cur.fetchall()]

    for _ in range(num_records):
        institution_id = random.choice(institution_ids)
        customer_id = random.choice(customer_ids)
        account_id = random.choice(account_ids)
        report_data = fake.text()
        report_date = fake.date_this_year()

        cur.execute("""
            INSERT INTO institution_reports (institution_id, customer_id, account_id, report_data, report_date)
            VALUES (%s, %s, %s, %s, %s)
        """, (institution_id, customer_id, account_id, report_data, report_date))

# Populate data
try:
    insert_customers(500)
    insert_financial_institutions(50)
    insert_credit_accounts(500)
    insert_payments(500)
    insert_credit_scores(500)
    insert_credit_reports(500)
    insert_public_records(100)
    insert_disputes(100)
    insert_loan_collateral(250)
    insert_institution_reports(500)

    # Commit all the transactions
    conn.commit()

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

finally:
    # Close the cursor and connection
    cur.close()
    conn.close()
